In [1]:

# ================================================
# 1️⃣ Env Prepare Install Required Packages
# ================================================

#%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):    
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@05b2c186c1b6c9a08375389d5efe9cb4c401c075#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo huggingface-hub==0.34.4 datasets==4.3.0 numpy==2.3.4 pandas==2.3.3 pyarrow==22.0.0 tqdm==4.67.1

Using Python 3.12.11 environment at: /home/zeus/miniconda3/envs/cloudspace
Resolved 11 packages in 19ms                                         
Audited 11 packages in 0.27ms


In [2]:
import torch
import bitsandbytes as bnb
from collections import Counter
from unsloth import FastLanguageModel


model, tokenizer = FastLanguageModel.from_pretrained(
    "unsloth/gpt-oss-20b",
#    load_in_4bit=True,
)

bnb_layers = []
for name, module in model.named_modules():
    if isinstance(module, bnb.nn.Linear4bit):
        bnb_layers.append(name)

print(f"BNB 4-bit layers found: {len(bnb_layers)}")
print("Sample BNB layer:", bnb_layers[0] if bnb_layers else "❌ NONE")
print("Model compute dtype:", model.dtype)



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.10: Fast Gpt_Oss patching. Transformers: 4.56.2.
   \\   /|    NVIDIA L40S. Num GPUs = 1. Max memory: 44.521 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

BNB 4-bit layers found: 1632
Sample BNB layer: model.layers.0.self_attn.q_proj
Model compute dtype: torch.bfloat16


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    checkpoint_path="ospost/gpt-oss-20b-sft-adapter"
)


In [ ]:
assert any("lora_" in n for n,_ in model.named_parameters())

lora_params = [
    (n, p) for n, p in model.named_parameters() if "lora_" in n
]

print(f"LoRA parameters found: {len(lora_params)}")
print("Sample LoRA param:", lora_params[0][0] if lora_params else "❌ NONE")

print("LoRA dtypes:", Counter(p.dtype for _, p in lora_params))

trainable = [
    n for n, p in model.named_parameters() if p.requires_grad
]

print(f"Trainable params count: {len(trainable)}")
print("Sample trainable param:", trainable[0] if trainable else "❌ NONE")



In [3]:
import os
os.environ["LD_LIBRARY_PATH"] = os.path.abspath("./llama.cpp") + ":" + os.environ.get("LD_LIBRARY_PATH", "")
# Optional: verify
print(os.environ["LD_LIBRARY_PATH"])

/teamspace/studios/this_studio/gpt-oss-20b-fine-tune/llama.cpp:/opt/amazon/efa/lib:/opt/aws-ofi-nccl/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


In [ ]:
import subprocess

res = subprocess.run(["./llama.cpp/llama-quantize", "--help"], capture_output=True, text=True)
print("Return code:", res.returncode)
print("Stdout:", res.stdout[:500])
print("Stderr:", res.stderr[:500])


In [ ]:
from unsloth_zoo.llama_cpp import check_llama_cpp

quantizer, converter = check_llama_cpp("./llama.cpp/")
print("Quantizer path:", quantizer)
print("Converter path:", converter)


In [4]:
model.save_pretrained("merged_model", tokenizer, save_method = "mxfp4",)

In [5]:
model.save_pretrained_gguf(
  "merged_model",
    tokenizer,
    quantization_method="q4_k_m"
)

Unsloth: Merging model weights to mxfp4 format...


/home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/unsloth_zoo/saving_utils.py:969: UserWarning: Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!
  warnings.warn("Model is not a PeftModel (no Lora adapters detected). Skipping Merge. Please use save_pretrained() or push_to_hub() instead!")


Unsloth: Converting to GGUF format...
Unsloth: GPT-OSS model detected - using special conversion settings
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF  might take 3 minutes.
\        /    [2] Converting GGUF  to ['None'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into  GGUF format.
This might take 3 minutes...


RuntimeError: Unsloth: GGUF conversion failed: Unsloth: Failed to convert model to GGUF: Command 'python llama.cpp/unsloth_convert_hf_to_gguf.py --outfile gpt-oss-20b.MXFP4.gguf --split-max-size 50G merged_model' returned non-zero exit status 1.

In [ ]:
!ls -lah gpt-oss-20b-base-gguf

